In [ ]:
import pandas as pd
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

In [ ]:
import pandas as pd
from product_mapping import product_id_mapping

# Replace 'data\hrm 6 66 d.xls' with the path to your actual Excel file
file_path = 'data\hrm 6 66 d.xls'
sheet_name = 'รายงานรายละเอียดการจ่ายเงินเดือ' 
df = pd.read_excel(file_path, skiprows=7, sheet_name=sheet_name, header=None)


# Define column names
column_names = ['Column1', 'Column2', 'Column3', 'Column4', 'Column5', 'Column6', 'Column7', 'Column8', 'Column9', 'Column10', 'Column11', 'Column12', 'Column13', 'Column14', 'Column15', 'Column16', 'Column17', 'Column18', 'Column19', 'Column20', 'Column21']

# Assign column names to the DataFrame
df.columns = column_names

# Display all columns without truncation and set precision for float columns
df = df.iloc[:-3]
df = df.fillna(0)
df[column_names[4:20]] = df[column_names[4:20]].astype(float)

pd.set_option('display.max_columns', None)
pd.set_option('display.float_format', '{:.2f}'.format)


# Replace '.' with empty string in Column1
df['Column1'] = df['Column1'].str.replace(r'^\.*', '', regex=True)
# drop column
df = df.drop(columns=['Column21'], axis=1)

new_columns = []
for col_num in range(4, 20):
    new_col_name = f'New_{column_names[col_num]}'
    df[new_col_name] = df[column_names[col_num]].shift(-1)


df['Column2'] = df['Column1'].map(lambda x: product_id_mapping.get(x, {}).get('Costcenter'))

df['Column5'] = df['Column5'] + df['New_Column5']
df['Column6'] = df['Column6'] + df['New_Column6']
df['Column7'] = df['Column7'] + df['New_Column7']+df['Column8'] + df['New_Column8']
# df['Column8'] = df['Column8'] + df['New_Column8']
df['Column9'] = df['Column9'] + df['New_Column9']
df['Column10'] = df['Column10'] + df['New_Column10']
df['Column12'] = df['New_Column11']+ df['Column12']+df['New_Column12']+df['Column13']
df['Column13']=df['New_Column13']
df['Column18']= df['New_Column15']+df['New_Column16']+df['New_Column17']+df['Column18']

columns_to_drop = ['Column4','Column8','New_Column5', 'New_Column6', 'New_Column7', 'New_Column8', 'New_Column9','New_Column10','New_Column11','New_Column12','New_Column13','New_Column14','New_Column15','New_Column16','New_Column17','New_Column19','New_Column20']

df = df.drop(columns=columns_to_drop, axis=1)

df['Column2'] = df['Column2'].apply(lambda x: '{:.0f}'.format(x) if isinstance(x, float) else x)

index=df.columns.get_loc('Column6')
df.insert(index, 'Column7', df.pop('Column7'))

index=df.columns.get_loc('Column12')
df.insert(index, 'Column13', df.pop('Column13'))

index=df.columns.get_loc('Column15')
df.insert(index, 'Column16', df.pop('Column16'))

index=df.columns.get_loc('Column16')
df.insert(index, 'New_Column18', df.pop('New_Column18'))

df.rename(columns={'Column1': 'ประเภท', 'Column2': 'CC.', 'Column3': 'คน', 'Column5': 'เงินเดือน/ค่าจ้าง', 'Column7': 'ตำแหน่ง/วิชา', 'Column6': 'ค่าครองชีพ','Column9': 'ค่าล่วงเวลา','Column10': 'เบี้ยขยัน','Column11': 'โบนัส','Column13': 'ค่าทักษะ','Column12': 'รายได้อื่นๆ','Column14': 'รวมรายได้','Column14': 'รวมรายได้', 'New_Column18':'ประกันสังคม','Column16': 'กองทุน','Column15': 'ภาษี','Column17': 'สหกรณ์','Column18': 'หักอื่นๆ','Column19': 'รวมค่าใช้จ่าย','Column20': 'รายได้สุทธิ'}, inplace=True)


df = df.iloc[::2]
df = df.fillna(0)
# Display all columns without truncation and set precision for float columns
output_file = 'outdata/output.xlsx'
df.to_excel(output_file, index=False)

# Print the formatted DataFrame
print(df)

                                      ประเภท      CC.  คน  เงินเดือน/ค่าจ้าง  \
0                        ส่วนออกแบบรูปลักษณ์  5010299   4           95415.00   
2                           หน่วยพัฒนาต้นแบบ  5010201   3           51810.00   
4             ส่วนวิเคราะห์และออกแบบวิศวกรรม  5010199  14          346560.00   
6                                  หน่วย PQC  4020201  18          194285.00   
8                                   หน่วย QC  4020202  27          295445.00   
10                          ส่วนประกันคุณภาพ  4020199   1           14245.00   
12        หน่วย คลังสินค้า-โรงประกอบ (04-64)  4010206  13          146690.00   
14  หน่วย คลังสินค้า-โรงผลิตชิ้นส่วน (04-64)  4010207  14          155365.00   
16      หน่วย Logistic 1 - Machinery (04-64)  4010301  10          102704.00   
18            หน่วย Logistic 2 - OEM (04-64)  4010302  15          147700.00   
20                                  หน่วย M2  8110302  17          159660.00   
22                                  หน่ว

In [ ]:
import pandas as pd
from product_mapping import product_id_mapping

fileP = 'outdata\output2023-08-28_17-29-30.xlsx'
df=pd.read_excel(fileP)

df['คน'] = df['ประเภท'].map(lambda x: product_id_mapping.get(x, {}).get('Costcenter'))

df_original = pd.DataFrame(df)

ids = df_original['ประเภท'].tolist()
ids_repeated = ids * 8

combined_data = []
for index, row in df_original.iterrows():
    combined_data.extend([row['เงินเดือน/ค่าจ้าง'], row['ค่าครองชีพ'], row['ตำแหน่ง/วิชา'], row['ค่าล่วงเวลา'], row['เบี้ยขยัน'], row['โบนัส'], row['รายได้อื่นๆ'], row['ค่าทักษะ']])

# loopCol = ['เงินเดือน/ค่าจ้าง','ค่าครองชีพ','ค่าล่วงเวลา']
# ids = ['ฝ่ายตรวจสอบภายใน และระบบงานคุณภาพ'] * 3  # Example IDs for demonstration
# column_names = ['เงินเดือน', 'ค่าครองชีพ', 'ค่าตำเเหน่ง']
# for i in loopCol:
#     print(i)
# id_value = 'ฝ่ายตรวจสอบภายใน และระบบงานคุณภาพ'
# column_name = 'Costcenter'
df_combined = pd.DataFrame({'ID': ids_repeated, 'Combined_Column': combined_data})
column_names=['เงินเดือน/ค่าจ้าง','ค่าครองชีพ','ตำแหน่ง/วิชา','ค่าล่วงเวลา','เบี้ยขยัน','โบนัส','รายได้อื่นๆ','ค่าทักษะ']
for id_value in ids_repeated:
    for column_name in column_names:
        data = product_id_mapping.get(id_value, {}).get(column_name)

print(df_combined)

                                 ID  Combined_Column
0               ส่วนออกแบบรูปลักษณ์            95415
1                  หน่วยพัฒนาต้นแบบ             3600
2    ส่วนวิเคราะห์และออกแบบวิศวกรรม            12500
3                         หน่วย PQC                0
4                          หน่วย QC              600
..                              ...              ...
195                       หน่วย บี1           162914
196                       หน่วย ซี4            33900
197                       หน่วย เอฟ                0
198              ส่วนผลิตชิ้นส่วน 2            72270
199                       หน่วย บี2            56170

[200 rows x 2 columns]


In [ ]:
import pandas as pd
from product_mapping import product_id_mapping

fileP = 'outdata\output2023-08-28_17-29-30.xlsx'
df = pd.read_excel(fileP)

# Map 'ประเภท' to 'Costcenter' and create a new column 'คน'
df['คน'] = df['ประเภท'].map(lambda x: product_id_mapping.get(x, {}).get('Costcenter'))

# Create a copy of the original DataFrame for reference
df_original = pd.DataFrame(df)

# Repeat the ids 8 times (assuming you have 8 repeated columns)
ids = df_original['ประเภท'].tolist()
ids_repeated = ids * 8

# Create a list of dictionaries to store the extended data
extended_data = []

# Loop through the original DataFrame and extend the data for each row
column_names=['เงินเดือน/ค่าจ้าง','ค่าครองชีพ','ตำแหน่ง/วิชา','ค่าล่วงเวลา','เบี้ยขยัน','โบนัส','รายได้อื่นๆ','ค่าทักษะ']
for index, row in df_original.iterrows():
    data_row = {
        
        'ID': row['ประเภท'],
        'Costcenter': product_id_mapping.get(row['ประเภท'], {}).get('Costcenter'),
        'Product': product_id_mapping.get(row['ประเภท'], {}).get('Product'),           
        }
    for col in column_names:
        data_row[col] = row[col]
    extended_data.extend([data_row] * 8)
        

# Create a new DataFrame using the extended data
new_df = pd.DataFrame(extended_data)
print(new_df)

# # Pivot the new DataFrame to get the desired format
# pivot_df = new_df.pivot(index='ID', columns=['Costcenter', 'Product'], values=['Value1', 'Value2', 'Value3', 'Value4'])

# # Reset the index
# pivot_df.reset_index(inplace=True)

# # Display the pivoted DataFrame
# print(pivot_df)


                      ID Costcenter Product  เงินเดือน/ค่าจ้าง  ค่าครองชีพ  \
0    ส่วนออกแบบรูปลักษณ์    5010299    1141              95415        3600   
1    ส่วนออกแบบรูปลักษณ์    5010299    1141              95415        3600   
2    ส่วนออกแบบรูปลักษณ์    5010299    1141              95415        3600   
3    ส่วนออกแบบรูปลักษณ์    5010299    1141              95415        3600   
4    ส่วนออกแบบรูปลักษณ์    5010299    1141              95415        3600   
..                   ...        ...     ...                ...         ...   
195            หน่วย บี2    7110301      00             799900       68400   
196            หน่วย บี2    7110301      00             799900       68400   
197            หน่วย บี2    7110301      00             799900       68400   
198            หน่วย บี2    7110301      00             799900       68400   
199            หน่วย บี2    7110301      00             799900       68400   

     ตำแหน่ง/วิชา  ค่าล่วงเวลา  เบี้ยขยัน  โบนัส  รายได้อื่นๆ  

In [ ]:
import pandas as pd
from product_mapping import product_id_mapping

fileP = 'outdata/admin2023-09-05_08-59-32.xlsx'
df = pd.read_excel(fileP)
tryit= df.shape[0]
print(tryit)
ids = df['ประเภท'].tolist()
ids_repeated = ids * 8
column_names=['เงินเดือน/ค่าจ้าง','ค่าครองชีพ','ตำแหน่ง/วิชา','ค่าล่วงเวลา','เบี้ยขยัน','โบนัส','รายได้อื่นๆ','ค่าทักษะ']
combined_data = []



for index, row in df.iterrows():
    combined_data.extend([row['เงินเดือน/ค่าจ้าง'], row['ค่าครองชีพ'], row['ตำแหน่ง/วิชา'], row['ค่าล่วงเวลา'], row['เบี้ยขยัน'], row['โบนัส'], row['รายได้อื่นๆ'], row['ค่าทักษะ']])



# Create the new combined DataFrame 
df_combined = pd.DataFrame({
    'Column1': ids_repeated,
    'DR.': combined_data,
})
value = product_id_mapping.get('ส่วนออกแบบรูปลักษณ์', {}).get('salary')
print(value)


for col in column_names:
    for i in range(tryit):
        df_combined['รหัสบัญชี'] = df_combined['Column1'].apply(lambda x: product_id_mapping.get(x, {}).get(col))

# c=df_combined['Column1'].apply(lambda x: product_id_mapping.get(x, {}).get('ค่าครองชีพ'))

# print(c)


df_combined['CC#'] = df_combined['Column1'].map(lambda x: product_id_mapping.get(x, {}).get('Costcenter'))
df_combined['Product'] = df_combined['Column1'].map(lambda x: product_id_mapping.get(x, {}).get('Product'))

columns_to_drop=['Column1']
df_combined = df_combined.drop(columns=columns_to_drop, axis=1)

index=df_combined.columns.get_loc('DR.')
df_combined.insert(index, 'รหัสบัญชี', df_combined.pop('รหัสบัญชี'))

index=df_combined.columns.get_loc('DR.')
df_combined.insert(index, 'CC#', df_combined.pop('CC#'))

index=df_combined.columns.get_loc('DR.')
df_combined.insert(index, 'Product', df_combined.pop('Product'))



print(df_combined)


14
None
     รหัสบัญชี      CC# Product     DR.
0      8211110  2049900      00  183880
1      8211110  2029900      00    4500
2      8211110  2019900      00   17300
3      8211110  2039900      00       0
4      8211110  1990000      00    1200
..         ...      ...     ...     ...
107    8211110  3029900      00       0
108    8211110  3019900      00       0
109    8211110  1029900      00       0
110    8211110  2069900      00       0
111    8211110  1990000      00       0

[112 rows x 4 columns]


In [ ]:
import pandas as pd
from product_mapping import product_id_mapping
value = product_id_mapping.get('ส่วนออกแบบรูปลักษณ์', {}).get('salary')
print(value)

8111101


In [ ]:
import glob

In [ ]:
print('ไทย')

ไทย


In [ ]:
import pandas as pd
from product_mapping import product_id_mapping

# Replace 'data\hrm 6 66 d.xls' with the path to your actual Excel file
file_path = 'data\direct.xls'

 
df = pd.read_excel(file_path, skiprows=7, header=None)


# Define column names
column_names = ['Column1', 'Column2', 'Column3', 'Column4', 'Column5', 'Column6', 'Column7', 'Column8', 'Column9', 'Column10', 'Column11', 'Column12', 'Column13', 'Column14', 'Column15', 'Column16', 'Column17', 'Column18', 'Column19', 'Column20', 'Column21']

df.columns = column_names

print(df)

                                              Column1  Column2       Column3  \
0                         ........ส่วนออกแบบรูปลักษณ์      NaN             4   
1                                                 NaN      NaN           NaN   
2                          ..........หน่วยพัฒนาต้นแบบ      NaN             3   
3                                                 NaN      NaN           NaN   
4              ........ส่วนวิเคราะห์และออกแบบวิศวกรรม      NaN            14   
5                                                 NaN      NaN           NaN   
6                                 ..........หน่วย PQC      NaN            19   
7                                                 NaN      NaN           NaN   
8                                  ..........หน่วย QC      NaN            27   
9                                                 NaN      NaN           NaN   
10                           ........ส่วนประกันคุณภาพ      NaN             1   
11                                      

In [ ]:
import pandas as pd
from product_mapping import product_id_mapping


file_path = 'data\indirect.xls'
df = pd.read_excel(file_path, skiprows=7, header=None)


column_names = ['Column1', 'Column2', 'Column3', 'Column4', 'Column5', 'Column6', 'Column7', 'Column8', 'Column9', 'Column10', 'Column11', 'Column12', 'Column13', 'Column14', 'Column15', 'Column16', 'Column17', 'Column18', 'Column19', 'Column20', 'Column21']

df.columns = column_names

df = df.iloc[:-3]
df = df.fillna(0)
df[column_names[4:20]] = df[column_names[4:20]].astype(float)

pd.set_option('display.max_columns', None)
pd.set_option('display.float_format', '{:.2f}'.format)


# Replace '.' with empty string in Column1
df['Column1'] = df['Column1'].str.replace(r'^\.*', '', regex=True)

df = df.drop(columns=['Column21'], axis=1)

new_columns = []
for col_num in range(4, 20):
    new_col_name = f'New_{column_names[col_num]}'
    df[new_col_name] = df[column_names[col_num]].shift(-1)

df = df.iloc[::2]
df['Column2'] = df['Column1'].map(lambda x: product_id_mapping.get(x, {}).get('Costcenter'))
df['รายได้อื่นๆ'] = df['Column12']+df['Column13'] +df['Column14']+df['Column15']+df['Column16']+df['Column17']
df['Column5'] = df['Column5'] + df['New_Column5']
df['Column6'] = df['Column6'] + df['New_Column6']
df['Column9'] = df['Column9'] + df['New_Column9']
df['Column10'] = df['Column10'] + df['New_Column10']
df['Column7'] = df['Column7'] + df['New_Column7'] + df['Column8'] + df['New_Column8']
df['Column8'] = df['Column5'] + df['Column6'] + df['Column7']+df['Column9']+df['Column10']+df['Column11']+df['Column12']+df['Column13']+df['Column14']+df['Column15']+df['Column16']+df['Column17']+df['Column18']



columns_to_drop = ['Column4','New_Column5','New_Column6','New_Column7','New_Column8','New_Column9','New_Column10','New_Column11','New_Column12','New_Column13','New_Column14','New_Column15','New_Column16','New_Column17','New_Column18','New_Column19','New_Column20']
df = df.drop(columns=columns_to_drop, axis=1)

index=df.columns.get_loc('Column18')
df.insert(index, 'Column8', df.pop('Column8'))

index=df.columns.get_loc('Column18')
df.insert(index, 'รายได้อื่นๆ', df.pop('รายได้อื่นๆ'))

df.rename(columns={'Column1': 'ประเภท', 'Column2': 'CC.', 'Column3': 'คน', 'Column5': 'เงินเดือน/ค่าจ้าง','Column6':'ค่าครองชีพ','Column7':'ตำแหน่ง/วิชา','Column8':'รวมรายได้','Column9':'ค่าล่วงเวลา','Column10':'เบี้ยขยัน','Column11':'โบนัส','Column12':'เงินรางวัล','Column13':'ค่าพาหนะ','Column14':'ค่ากะ','Column15':'เงินชดเชย','Column16':'ค่าเบี้ยเลี้ยง','Column17':'รับอื่นๆ','Column18':'ค่าทักษะ','Column19':'กองทุน','Column20':'ประกันสังคม'},inplace=True)



output_file = 'testOut/1test.xlsx'
df.to_excel(output_file, index=False)


print(df)

                                 ประเภท      CC.  คน  เงินเดือน/ค่าจ้าง  \
0   Thairung Engineering Center (04-64)  5990000   6          198400.00   
2                     ฝ่ายวิจัยและพัฒนา  5019900   6          196900.00   
4                 ฝ่ายวิศวกรรมการประกอบ  5049900  15          243210.00   
6              ฝ่ายวิศวกรรมผลิตชิ้นส่วน  5039900  21          327235.00   
8     ด้านการผลิต Manufacturing (04-64)  4990000  12          158035.00   
10                     ฝ่ายประกันคุณภาพ  4029900  17          345344.00   
12           ฝ่ายวางแผนและควบคุมการผลิต  4019900  32          582150.00   
14                          ฝ่ายการผลิต  8119900  22          463815.00   
16                     ฝ่ายผลิตชิ้นส่วน  7119900  31          462767.00   
18                   ฝ่ายวิศวกรรมโรงงาน  5029900  26          530610.00   

    ค่าครองชีพ  ตำแหน่ง/วิชา  ค่าล่วงเวลา  เบี้ยขยัน  โบนัส  เงินรางวัล  \
0      3600.00      31000.00      2291.00     500.00   0.00        0.00   
2      3600.00      2390

In [ ]:
import pandas as pd

from product_mapping import product_id_mapping


file_path = 'testOut/1test.xlsx'
df = pd.read_excel(file_path,)

column_names=['เงินเดือน/ค่าจ้าง','ค่าครองชีพ','ตำแหน่ง/วิชา','ค่าล่วงเวลา','เบี้ยขยัน','โบนัส','รายได้อื่นๆ','ค่าทักษะ']



for i,n in enumerate(column_names):
    
    df1 = df[['ประเภท','CC.',n]]
    df1['บัญชี'] = df['ประเภท'].map(lambda x: product_id_mapping.get(x, {}).get(n))
    df1.rename(columns={n:'นั้นเเหละครับ'})


C:\Users\ITSanthiti\AppData\Local\Temp\ipykernel_1280\335793632.py:15: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df1['บัญชี'] = df['ประเภท'].map(lambda x: product_id_mapping.get(x, {}).get(n))
C:\Users\ITSanthiti\AppData\Local\Temp\ipykernel_1280\335793632.py:15: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df1['บัญชี'] = df['ประเภท'].map(lambda x: product_id_mapping.get(x, {}).get(n))
C:\Users\ITSanthiti\AppData\Local\Temp\ipykernel_1280\335793632.py:15: SettingWithCopyWarning: 
A value is trying to b

In [ ]:
import pandas as pd
from product_mapping import product_id_mapping

file_path = 'testOut/1.xlsx'
df = pd.read_excel(file_path)

pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)
# pd.set_option('display.float_format', '{:.2f}'.format)

column_names = ['เงินเดือน/ค่าจ้าง', 'ค่าครองชีพ', 'ตำแหน่ง/วิชา', 'ค่าล่วงเวลา', 'เบี้ยขยัน', 'โบนัส', 'รายได้อื่นๆ', 'ค่าทักษะ']

# Initialize an empty list to store the dataframes
dfs_to_concat = []

for i, n in enumerate(column_names):
    # Create a new dataframe for each column
    df1 = df[['ประเภท', 'CC.', n]].copy()  # Use .copy() to create a copy of the DataFrame
    df1['บัญชี'] = df['ประเภท'].map(lambda x: product_id_mapping.get(x, {}).get(n))
    
    # Rename the column 'n' to 'เงิน'
    df1.rename(columns={n: 'เงิน'}, inplace=True)
    
    # Append the dataframe to the list
    dfs_to_concat.append(df1)

# Concatenate the dataframes vertically
result_df = pd.concat(dfs_to_concat)

result_df=result_df.drop(columns=['ประเภท'],axis=1)

# df = df.drop(columns=['Column21'], axis=1)
# result_df.to_excel('testOut/1out.xlsx', index=False)

# Now, result_df contains the concatenated data
print(result_df)


        CC.     เงิน      บัญชี
0   2049900   183880 8111101.00
1   2029900   364579 8111101.00
2   2019900   630280 8111101.00
3   2039900   265925 8111101.00
4   1990000  1793920 8111101.00
5   2059900   725800 8111101.00
6   2079900    70265 8111101.00
7   2069900   126585 8111101.00
8   3990000   190005 8111101.00
9   3029900   440070 8111101.00
10  3019900   117745 8111101.00
11  1029900    47635 8111101.00
12  2069900   192960 8111101.00
13  1990000    50000 8111101.00
14      รวม  5199649        NaN
0   2049900     4500 8211102.00
1   2029900     8996 8211102.00
2   2019900    15300 8211102.00
3   2039900     7200 8211102.00
4   1990000     8562 8211102.00
5   2059900    28800 8211102.00
6   2079900     1800 8211102.00
7   2069900     2700 8211102.00
8   3990000      900 8211102.00
9   3029900     8280 8211102.00
10  3019900     3600 8211102.00
11  1029900      900 8211102.00
12  2069900     2700 8211102.00
13  1990000        0 8211102.00
14      รวม    94238        NaN
0   2049

In [ ]:
import pandas as pd

# Sample DataFrames
df1 = pd.DataFrame({'A': [1, 2, 3], 'B': [4, 5, 6]})
df2 = pd.DataFrame({'A': [7, 8, 9], 'B': [10, 11, 12]})

# Concatenate vertically
result = pd.concat([df1, df2], axis=0)

print(result)


   A   B
0  1   4
1  2   5
2  3   6
0  7  10
1  8  11
2  9  12


In [ ]:
import pandas as pd
from product_mapping import product_id_mapping
import glob
import os

folder_path = "data/sep"
file_paths = glob.glob(os.path.join(folder_path, '*.xls'))

for fileN,file_path in enumerate(file_paths):
    df = pd.read_excel(file_path, skiprows=7,header=None)
    
    column_names = ['Column1', 'Column2', 'Column3', 'Column4', 'Column5', 'Column6', 'Column7', 'Column8', 'Column9', 'Column10', 'Column11', 'Column12', 'Column13', 'Column14', 'Column15', 'Column16', 'Column17', 'Column18', 'Column19', 'Column20', 'Column21']
    df.columns = column_names

    df = df.iloc[:-3]
    df = df.fillna(0)
    df[column_names[4:20]] = df[column_names[4:20]].astype(float)

    pd.set_option('display.max_columns', None)
    pd.set_option('display.float_format', '{:.2f}'.format)


    # Replace '.' with empty string in Column1
    df['Column1'] = df['Column1'].str.replace(r'^\.*', '', regex=True)

    df = df.drop(columns=['Column21'], axis=1)

    new_columns = []
    for col_num in range(4, 20):
        new_col_name = f'New_{column_names[col_num]}'
        df[new_col_name] = df[column_names[col_num]].shift(-1)

    df = df.iloc[::2]
    df['Column2'] = df['Column1'].map(lambda x: product_id_mapping.get(x, {}).get('Costcenter'))
    df['รายได้อื่นๆ'] = df['Column12']+df['Column13'] +df['Column14']+df['Column16']+df['Column17']
    df['Column5'] = df['Column5'] + df['New_Column5']
    df['Column6'] = df['Column6'] + df['New_Column6']
    df['Column7'] = df['Column7'] + df['New_Column7'] + df['Column8'] + df['New_Column8']
    df['Column9'] = df['Column9'] + df['New_Column9']
    df['Column10'] = df['Column10'] + df['New_Column10']

    df['Column8'] = df['Column5'] + df['Column6'] + df['Column7']+df['Column9']+df['Column10']+df['Column11']+df['Column12']+df['Column13']+df['Column14']+df['Column15']+df['Column16']+df['Column17']+df['Column18']


    columns_to_drop = ['Column4','New_Column5','New_Column6','New_Column7','New_Column8','New_Column9','New_Column10','New_Column11','New_Column12','New_Column13','New_Column14','New_Column15','New_Column16','New_Column17','New_Column18','New_Column19','New_Column20']
    df = df.drop(columns=columns_to_drop, axis=1)

    index=df.columns.get_loc('Column18')
    df.insert(index, 'Column8', df.pop('Column8'))

    index=df.columns.get_loc('Column18')
    df.insert(index, 'รายได้อื่นๆ', df.pop('รายได้อื่นๆ'))

    df.rename(columns={'Column1': 'ประเภท', 'Column2': 'CC.', 'Column3': 'คน', 'Column5': 'เงินเดือน/ค่าจ้าง','Column6':'ค่าครองชีพ','Column7':'ตำแหน่ง/วิชา','Column8':'รวมรายได้','Column9':'ค่าล่วงเวลา','Column10':'เบี้ยขยัน','Column11':'โบนัส','Column12':'เงินรางวัล','Column13':'ค่าพาหนะ','Column14':'ค่ากะ','Column15':'เงินชดเชย','Column16':'ค่าเบี้ยเลี้ยง','Column17':'รับอื่นๆ','Column18':'ค่าทักษะ','Column19':'กองทุน','Column20':'ประกันสังคม'},inplace=True)

# Create a new row with the total people
    total_row = pd.DataFrame({'CC.': ['รวม'], 'คน': [df['คน'].sum()], 'เงินเดือน/ค่าจ้าง': [df['เงินเดือน/ค่าจ้าง'].sum()], 'ค่าครองชีพ': [df['ค่าครองชีพ'].sum()], 'ตำแหน่ง/วิชา': [df['ตำแหน่ง/วิชา'].sum()], 'รวมรายได้': [df['รวมรายได้'].sum()], 'ค่าล่วงเวลา': [df['ค่าล่วงเวลา'].sum()], 'เบี้ยขยัน': [df['เบี้ยขยัน'].sum()], 'โบนัส': [df['โบนัส'].sum()], 'เงินรางวัล': [df['เงินรางวัล'].sum()], 'ค่าพาหนะ': [df['ค่าพาหนะ'].sum()], 'ค่ากะ': [df['ค่ากะ'].sum()], 'เงินชดเชย': [df['เงินชดเชย'].sum()], 'ค่าเบี้ยเลี้ยง': [df['ค่าเบี้ยเลี้ยง'].sum()], 'รับอื่นๆ': [df['รับอื่นๆ'].sum()], 'ค่าทักษะ': [df['ค่าทักษะ'].sum()], 'รายได้อื่นๆ': [df['รายได้อื่นๆ'].sum()], 'กองทุน': [df['กองทุน'].sum()], 'ประกันสังคม': [df['ประกันสังคม'].sum()]})
    
    df = pd.concat([df, total_row], ignore_index=True)

    # Create a new DataFrame with the total_row
    df.to_excel('holder/{}.xlsx'.format(fileN),index=False)

In [ ]:
def keep_first_o_i_t(lst):
    
    found = {'เงินเดือน/ค่าจ้าง': False, 'ค่าครองชีพ': False, 'ตำแหน่ง/วิชา': False, 'ค่าล่วงเวลา': False, 'เบี้ยขยัน': False, 'โบนัส': False, 'รายได้อื่นๆ': False, 'ค่าทักษะ': False, 'เงินชดเชย': False}
    result = []
    
    for val in lst:
        if val in found:
            if not found[val]:
                found[val] = True
                result.append(val)
            else:
                result.append('')
        else:
            result.append('')
    
    return result
    
    

In [ ]:
import pandas as pd
from product_mapping import product_id_mapping
import glob
import os

folder_path = 'holder'
file_paths = glob.glob(os.path.join(folder_path, '*.xlsx'))

column_names = ['เงินเดือน/ค่าจ้าง', 'ค่าครองชีพ', 'ตำแหน่ง/วิชา', 'ค่าล่วงเวลา', 'เบี้ยขยัน', 'โบนัส', 'รายได้อื่นๆ', 'ค่าทักษะ','เงินชดเชย']
all_dataframes = [] 
conca2=[]
conca3=[]

for file_path in file_paths:
    df = pd.read_excel(file_path,)
    df = df.iloc[:-1]

    dfs_to_concat = [] # List to hold dataframes for each column

    for column_name in column_names:
        df1 = df[['ประเภท', 'CC.', column_name]].copy()
        df1['ชื่อบัญชี']=column_name
        df1['บัญชี'] = df['ประเภท'].map(lambda x: product_id_mapping.get(x, {}).get(column_name))
        df1.rename(columns={column_name: 'จำนวนเงิน'}, inplace=True)
        df1['prod.'] = df['ประเภท'].map(lambda x: product_id_mapping.get(x, {}).get('Product'))
        index=df1.columns.get_loc('CC.')
        df1.insert(index, 'บัญชี', df1.pop('บัญชี'))
        index = df1.columns.get_loc('จำนวนเงิน')
        df1.insert(index, 'prod.', df1.pop('prod.'))
        index = df1.columns.get_loc('ประเภท')
        df1.insert(index, 'ชื่อบัญชี', df1.pop('ชื่อบัญชี'))
        dfs_to_concat.append(df1)
        
        
    # Concatenate dataframes for this file and append to all_dataframes
    result_df = pd.concat(dfs_to_concat)
    all_dataframes.append(result_df)
    
    df2 = df[['ประเภท', 'CC.', 'กองทุน']].copy()
    df2['ชื่อบัญชี'] = 'กองทุน'
    df2['บัญชี'] = df['ประเภท'].map(lambda x: product_id_mapping.get(x, {}).get('กองทุน'))
    df2.rename(columns={'กองทุน': 'จำนวนเงิน'}, inplace=True)
    df2['prod.'] = df['ประเภท'].map(lambda x: product_id_mapping.get(x, {}).get('Product'))
    index=df2.columns.get_loc('CC.')
    df2.insert(index, 'บัญชี', df2.pop('บัญชี'))
    index = df2.columns.get_loc('จำนวนเงิน')
    df2.insert(index, 'prod.', df2.pop('prod.'))
    index = df2.columns.get_loc('ประเภท')
    df2.insert(index, 'ชื่อบัญชี', df2.pop('ชื่อบัญชี'))
    conca2.append(df2)

    df3 = df[['ประเภท', 'CC.', 'ประกันสังคม']].copy()
    df3['ชื่อบัญชี'] = 'ประกันสังคม'
    df3['บัญชี'] = df['ประเภท'].map(lambda x: product_id_mapping.get(x, {}).get('ประกันสังคม'))
    df3.rename(columns={'ประกันสังคม': 'จำนวนเงิน'}, inplace=True)
    df3['prod.'] = df['ประเภท'].map(lambda x: product_id_mapping.get(x, {}).get('Product'))
    index=df3.columns.get_loc('CC.')
    df3.insert(index, 'บัญชี', df3.pop('บัญชี'))
    index = df3.columns.get_loc('จำนวนเงิน')
    df3.insert(index, 'prod.', df3.pop('prod.'))
    index = df3.columns.get_loc('ประเภท')
    df3.insert(index, 'ชื่อบัญชี', df3.pop('ชื่อบัญชี'))
    conca3.append(df3)
    
soc = pd.concat(conca3, ignore_index=True)   
pf = pd.concat(conca2 , ignore_index=True) 
final = pd.concat(all_dataframes, ignore_index=True)

template =pd.read_excel('template.xlsx')
cce = pd.concat([final, template], ignore_index=True)

cce.to_excel('test/นี้.xlsx',index=False)
pf.to_excel('test/pf.xlsx',index=False)
soc.to_excel('test/soc.xlsx', index=False)
    
    


In [ ]:
import pandas as pd
import glob
import os

# Define the file paths for your Excel files
excel_file_paths = 'testOut'
file_path= glob.glob(os.path.join(excel_file_paths,'*.xlsx'))

# Initialize an empty list to store DataFrames
dfs = []

# Loop through each Excel file
for excel_file_path in file_path:
    # Read data from the Excel file into a DataFrame
    df = pd.read_excel(excel_file_path)
    
    # Append the DataFrame to the list
    dfs.append(df)

# Concatenate all DataFrames vertically into one DataFrame
merged_df = pd.concat(dfs, ignore_index=True)

merged_df.to_excel('try.xlsx',index=False)

# Now, merged_df contains the combined data from all three Excel files


In [ ]:
import shutil
from datetime import datetime
current_datetime = datetime.now()
# formatted_datetime = current_datetime.strftime("%Y-%m-%d %H:%M:%S")
directory='outfinal/sep_{}'.format(formatted_datetime)
print(directory)
if os.path.exists(directory):
                # If it exists, remove the directory and its contents
    shutil.rmtree(directory)
os.makedirs(directory)
print(f"Directory '{directory}' created or overwritten successfully.")

outfinal/sep_2023-09-11 16:38:48


OSError: [WinError 123] The filename, directory name, or volume label syntax is incorrect: 'outfinal/sep_2023-09-11 16:38:48'

In [ ]:
import shutil
from datetime import datetime

current_datetime = datetime.now()
formatted_datetime = current_datetime.strftime("%Y-%m-%d_%H-%M-%S")  # Use underscores instead of colons
directory = f'outfinal/sep_{formatted_datetime}'  # Use forward slash or backslash as needed

if os.path.exists(directory):
    # If it exists, remove the directory and its contents
    shutil.rmtree(directory)

os.makedirs(directory)
print(f"Directory '{directory}' created or overwritten successfully.")


Directory 'outfinal/sep_2023-09-11_16-39-23' created or overwritten successfully.


In [ ]:
import openpyxl
from openpyxl.styles import Font
import glob

folder_path = 'data/sep1'
file_paths = glob.glob(os.path.join(folder_path, '*.xls'))

for file in file_path:
    wb = openpyxl.load_workbook(file)
    font_style = Font(name='Calibri',size=11)
    wb.style = font_style
    wb.save(file) 


InvalidFileException: openpyxl does not support  file format, please check you can open it with Excel first. Supported formats are: .xlsx,.xlsm,.xltx,.xltm

In [ ]:
import openpyxl
from openpyxl.styles import Font
import glob

folder_path = 'data/sep1'
file_paths = glob.glob(os.path.join(folder_path, '*.xls'))

for file_path in file_paths:
    wb = openpyxl.load_workbook(file_path)

    # Iterate through all sheets in the workbook
    for sheet_name in wb.sheetnames:
        sheet = wb[sheet_name]

        # Iterate through all cells in the sheet
        for row in sheet.iter_rows():
            for cell in row:
                # Apply the font style to the cell
                cell.font = Font(name='Calibri', size=11)

    # Save the modified workbook
    wb.save(file_path)

InvalidFileException: openpyxl does not support the old .xls file format, please use xlrd to read this file, or convert it to the more recent .xlsx file format.

In [ ]:
import pandas as pd

file = 'template.xlsx'
df = pd.read_excel(file)
print(df)

                     ชื่อบัญชี  \
0              C/A SCB ซอย 114   
1              C/A SCB ซอย 114   
2                เจ้าหนี้อื่นๆ   
3                เจ้าหนี้อื่นๆ   
4                เจ้าหนี้อื่นๆ   
5                เจ้าหนี้อื่นๆ   
6                เจ้าหนี้อื่นๆ   
7                เจ้าหนี้อื่นๆ   
8                เจ้าหนี้อื่นๆ   
9                เจ้าหนี้อื่นๆ   
10  ภาษีหัก ณ ที่จ่าย ค้างจ่าย   
11             เงินประกันสังคม   
12    เงินกองทุนสำรองเลี้ยงชีพ   
13          สวัสดิการสงเคราะห์   
14          เงินสหกรณ์ค้างจ่าย   
15     ลูกหนี้เงินเบิกล่วงหน้า   
16                ลูกหนี้อื่นๆ   
17                ลูกหนี้อื่นๆ   
18                ลูกหนี้อื่นๆ   
19     รายได้อื่นๆ - ไม่มี vat   
20    รายได้ชดเชย - ค่าเสียหาย   
21           เงินฝาก S/A - SCB   
22           เงินฝาก S/A - SCB   
23           เงินฝาก S/A - SCB   

                                               ประเภท    บัญชี  CC.  prod.  \
0                   โอนเงินเดือน > SCB ประจำรอบ…...คน  1114011  NaN    

In [1]:
import os
folder_path = input('Pls type a directory name :')
if os.path.exists('data\{}'.format(folder_path)):
    print('sdsd')
else:
    print('sdsdeu4o2')

sdsdeu4o2


In [7]:
def number_to_string(argument):
    match argument:
        case 0:
            return "zero"
        case 1:
            return "one"
        case 2:
            return "two"
        case default:
            raise ValueError("Invalid choice. Program will stop.")
  
try:
    choice = input()
    result = number_to_string(choice)
    print(f"The result for choice {choice} is: {result}")
except ValueError as e:
    print(e)
print('sdsd')

Invalid choice. Program will stop.
sdsd


In [4]:
from mapping.Trt_mapping import product_id_mapping_Trt
product_id_mapping = product_id_mapping_Trt

package3 = 'mapping.Trt_mapping' in globals()

if package3:
    print('true')